## IMDB scraping

In [22]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
import re
from time import sleep
from random import randint
import pandas as pd

C:\Users\victo\AppData\Local\Temp\ipykernel_6340\3068543813.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


- Create a MSQL database connection for storing the data

In [3]:
try:
    connection = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'qwerty',
    database = 'imdb',
    port = 3306
)

except Exception as e:
    print(e)
else:
    print('Connection Successful')

Connection Successful


In [20]:
base_url = "https://www.imdb.com"
page_url = "https://www.imdb.com/chart/top/"
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}

response = requests.get(page_url,headers=headers)
imdb_soup = BeautifulSoup(response.text, "html.parser")
movies = imdb_soup.find_all('li', attrs= {'class': 'ipc-metadata-list-summary-item'})


<Response [200]>


In [12]:
first_movie_link = movies[2]
first_movie_link = first_movie_link.find('a', attrs={'class': 'ipc-title-link-wrapper'})['href']
first_movie_link

'/title/tt0468569/?ref_=chttp_t_3'

- Function for converting duration to minutes

In [13]:
#Duration converter to minutes:
def duration_to_minutes(duration_str):
    #Split the duration string by 'h' and 'm'
    
    minutes = 0
    parts = duration_str.split()
    
    for part in parts:
        if 'h' in part:
            hours = int(part.strip('h'))
            minutes = hours*60
        elif 'm' in part:
            mins = int(part.strip('m'))
            minutes+=mins
    return minutes
duration_to_minutes('1h 3m')

63

- Create a cursor for entering data into database

In [6]:
cursor = connection.cursor()
data_entry_query = f'INSERT INTO top_250 (movie_id, movie_title, year_released, audience, duration,rating, director,stars) VALUES (%s, %s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE movie_id = VALUES(movie_id), movie_title = VALUES(movie_title), year_released = VALUES(year_released), audience = VALUES(audience),duration = VALUES(duration), rating = VALUES(rating),director = VALUES(director),stars = VALUES(stars);'

In [ ]:
for movie in movies:
    movie_link = movie.find('a', attrs={'class': 'ipc-title-link-wrapper'})['href']
    complete_link = base_url+movie_link
    print(complete_link)

    #Waiting time between requests
    # sleep(randint(2,5))
    try:
        response = requests.get(complete_link, headers=headers)
    
        movie_soup = BeautifulSoup(response.text, 'html.parser')
        movie_id = re.search(r'tt\d{7}', complete_link).group()
        movie_title = movie_soup.find('span', attrs={'class': 'hero__primary-text'}).text
        year_released = movie_soup.find('ul', attrs={'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'}).contents[0].text
        audience = movie_soup.find('ul', attrs={'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'}).contents[1].text
        duration = movie_soup.find('ul', attrs={'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'}).contents[2].text
        duration  = duration_to_minutes(duration)
        rating = movie_soup.find('span', attrs={'class': 'sc-bde20123-1 cMEQkK'}).text
        director = movie_soup.find('a', attrs={'class': 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text
        stars_soup = movie_soup.find('li', attrs={'role':"presentation", 'class':"ipc-metadata-list__item ipc-metadata-list-item--link", 'data-testid':"title-pc-principal-credit"})
        stars = ''
        for star in stars_soup.find_all('a', class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"):
            stars+=star.text + ', '
            
        # Remove the trailing comma and space
        stars = stars.rstrip(', ')

        print(movie_id,movie_title,year_released, audience, duration, rating, director, stars)

        values = (movie_id,movie_title,year_released, audience, duration, rating, director, stars)
        cursor.execute(data_entry_query, values)
        connection.commit()
    except Exception as e:
        print(f"status code: {response.status_code}")
        print(e)


In [24]:
sql_query = "SELECT * FROM top_250"
df = pd.read_sql(sql_query, connection)
df.head()

C:\Users\victo\AppData\Local\Temp\ipykernel_6340\3446054449.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, connection)


,movie_id,movie_title,year_released,audience,duration,rating,director,stars
0,tt0012349,The Kid,1921,Passed,68,8.2,Charles Chaplin,"Charles Chaplin, Edna Purviance, Jackie Coogan"
1,tt0015324,Sherlock Jr.,1924,Passed,45,8.2,Buster Keaton,"Buster Keaton, Kathryn McGuire, Joe Keaton"
2,tt0015864,The Gold Rush,1925,Passed,95,8.1,Charles Chaplin,"Charles Chaplin, Mack Swain, Tom Murray"
3,tt0017136,Metropolis,1927,Not Rated,153,8.3,Fritz Lang,"Thea von Harbou, Fritz Lang"
4,tt0017925,The General,1926,Passed,78,8.1,Clyde Bruckman,"Buster Keaton, Clyde Bruckman, Al Boasberg"


In [26]:
df.shape


(246, 8)

In [27]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
year_released,246.0,1987.203252,25.608021,1921.0,1967.25,1995.0,2007.00,2024.0
duration,246.0,129.739837,29.861814,45.0,109.00,127.0,145.75,238.0
rating,246.0,8.310976,0.236128,8.0,8.10,8.2,8.40,9.3


In [28]:
df.isnull().sum()

movie_id         0
movie_title      0
year_released    0
audience         0
duration         0
rating           0
director         0
stars            0
dtype: int64

In [31]:
df.duplicated().sum()

0